### Importing libraries

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup

print('Libraries imported.')
%matplotlib inline

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

###  Importing dataframe 

csv from https://simplemaps.com/data/no-cities imported through IBM Studio

### Data cleaning

In [ ]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

In [3]:
df = pd.read_csv('no.csv')
df.head()

,city,lat,lng,country,iso2,admin,capital,population,population_proper
0,Oslo,59.916667,10.750000,Norway,NO,Oslo,primary,835000.0,580000.0
1,Bergen,60.391111,5.324722,Norway,NO,Hordaland,admin,213585.0,187194.0
2,Stavanger,58.966667,5.750000,Norway,NO,Rogaland,admin,173132.0,100866.0
3,Trondheim,63.416667,10.416667,Norway,NO,Sør-Trøndelag,admin,147139.0,141533.0
4,Drammen,59.742500,10.211667,Norway,NO,Buskerud,admin,90722.0,80153.0


In [4]:
df.sort_values(by = 'population', ascending = False)
df.drop(columns = ['country', 'iso2', 'admin', 'capital'], inplace = True)
df = df.rename(columns = {'lat':'Latitude', 'lng': 'Longitude', 'city': 'City', 'population': 'Population', 'population_proper': 'Population_proper'})
df.head()

,City,Latitude,Longitude,Population,Population_proper
0,Oslo,59.916667,10.750000,835000.0,580000.0
1,Bergen,60.391111,5.324722,213585.0,187194.0
2,Stavanger,58.966667,5.750000,173132.0,100866.0
3,Trondheim,63.416667,10.416667,147139.0,141533.0
4,Drammen,59.742500,10.211667,90722.0,80153.0


In [5]:
# Removing population columns now that the data is sorted
df = df.drop(columns= ['Population', 'Population_proper'])
df.head()

,City,Latitude,Longitude
0,Oslo,59.916667,10.750000
1,Bergen,60.391111,5.324722
2,Stavanger,58.966667,5.750000
3,Trondheim,63.416667,10.416667
4,Drammen,59.742500,10.211667


### Creating maps

In [6]:
address = 'Norway'
geolocator = Nominatim(user_agent="norway")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Norway are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Norway are 64.5731537, 11.52803643954819.


In [7]:
map_norway = folium.Map(location=[latitude, longitude], zoom_start=7)

# adding markers to the map
for lat, lng, city in zip(df['Latitude'], df['Longitude'], df['City']):
    label = ' {}'.format(city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_norway)  
    
map_norway

### Connecting to Foursquare

In [8]:
CLIENT_ID = 'TWJIJVESXEQPOBHWYA53BXI3RLRA0XK0BP132CKNV4XTJDHX' 
CLIENT_SECRET = 'IGQD05EBWWDCFUUXOQN5BLA2KJUO4OCXMBNFOJ30QLLIBOBZ' 
VERSION = '20180605' 
LIMIT = 100

In [9]:
#Function to apply it to every Postcode we have.
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
norway_venues = getNearbyVenues(names=df['City'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Oslo
Bergen
Stavanger
Trondheim
Drammen
Skyer
Kristiansand
Tomasjorda
Ålesund
Sandnes
Haugesund
Tønsberg
Moss
Bodø
Arendal
Hamar
Gjøvik
Mo i Rana
Narvik
Lillehammer
Harstad
Molde
Alta
Steinkjer
Hammerfest
Namsos
Vossevangen
Vadsø
Svolvær
Finnsnes
Kirkenes
Rørvik
Hermansverk
Skogsvåg
Noresund
Sandnessjøen
Hansnes
Råde
Tingvoll
Årdalstangen
Rosendal
Batnfjordsøra
Namdalseid
Jaren
Hov
Kragerø
Stranda
Gratangen
Valldal
Ulefoss
Våler
Drøbak
Hokksund
Lurøy
Vik
Tysse
Kleppe
Skjåk
Vågå
Åseral
Langesund
Sørumsand
Randaberg
Treungen
Ølen
Sykkylven
Ål
Kongsberg
Ski
Dale
Lyngdal
Kongsvinger
Brøstadbotn
Sør-Fron
Oppdal
Bjørkelangen
Alvdal
Brekstad
Iveland
Storslett
Søgne
Rena
Evje
Brønnøysund
Selbu
Nesna
Nannestad
Andenes
Borkenes
Bremnes
Jevnaker
Bruflat
Stokmarknes
Kjøllefjord
Røyrvik
Fiskå
Rognan
Hommelvik
Volda
Alstad
Valderhaug
Surnadal
Hauge
Bø
Gryllefjord
Kopervik
Gjerstad
Leksvik
Kvalsund
Ås
Reine
Brattvåg
Støren
Meldal
Larsnes
Lena
Sørland
Steinshamn
Skjervøy
Skarnes
Dokka
Masfjordnes
Kvin

In [31]:
print(norway_venues.shape)
norway_venues.head()

(1163, 7)


,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Oslo,59.916667,10.75,Torggata Botaniske,59.916070,10.752173,Cocktail Bar
1,Oslo,59.916667,10.75,BRUS Bar,59.916625,10.753520,Beer Bar
2,Oslo,59.916667,10.75,Crowbar & Bryggeri,59.917204,10.753523,Brewery
3,Oslo,59.916667,10.75,Arakataka,59.916381,10.750594,Scandinavian Restaurant
4,Oslo,59.916667,10.75,My Ugly Baby,59.915015,10.750212,Coffee Shop


In [32]:
norway_venues.groupby('City').count().head()

,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,
Akkerhaugen,2,2,2,2,2,2
Alstad,2,2,2,2,2,2
Alta,4,4,4,4,4,4
Andenes,1,1,1,1,1,1
Arendal,22,22,22,22,22,22


In [33]:
print('There are {} uniques categories.'.format(len(norway_venues['Venue Category'].unique())))

There are 197 uniques categories.


In [34]:
# one hot encoding
norway_onehot = pd.get_dummies(norway_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
norway_onehot['City'] = norway_venues['City'] 

# move neighborhood column to the first column
fixed_columns = [norway_onehot.columns[-1]] + list(norway_onehot.columns[:-1])
norway_onehot = norway_onehot[fixed_columns]
norway_onehot.head()

,City,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bakery,Bar,Bathing Area,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Boat or Ferry,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Camera Store,Campground,Canal,Child Care Service,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Disc Golf,Dog Run,Dry Cleaner,Electronics Store,Factory,Falafel Restaurant,Farm,Fast Food Restaurant,Fish Market,Fishing Spot,Fishing Store,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Forest,French Restaurant,Fruit & Vegetable Store,Furniture / Home Store,Garden Center,Gas Station,Gastropub,General Entertainment,General Travel,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Herbs & Spices Store,Historic Site,History Museum,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Lake,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Mini Golf,Miscellaneous Shop,Motel,Motorsports Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,New American Restaurant,Nightclub,Opera House,Other Repair Shop,Outlet Store,Park,Pedestrian Plaza,Performing Arts Venue,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Racetrack,Ramen Restaurant,Real Estate Office,Record Shop,Recreation Center,Rental Car Location,Resort,Restaurant,River,Rock Club,Romanian Restaurant,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Area,Soccer Field,Soccer Stadium,Soup Place,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Thai Restaurant,Theater,Theme Park,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Travel & Transport,Tree,Tunnel,Vacation Rental,Vegetarian / Vegan Restaurant,Watch Shop,Water Park,Waterfall,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Zoo
0,Oslo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Oslo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Oslo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Oslo,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [35]:
norway_onehot.shape

(1163, 198)

In [36]:
#Here we are showing the mean of what each neighborhood has close by
norway_grouped = norway_onehot.groupby('City').mean().reset_index()
norway_grouped.head()

,City,American Restaurant,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bakery,Bar,Bathing Area,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Boat or Ferry,Bookstore,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Cafeteria,Café,Camera Store,Campground,Canal,Child Care Service,Chinese Restaurant,Clothing Store,Cocktail Bar,Coffee Roaster,Coffee Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Diner,Disc Golf,Dog Run,Dry Cleaner,Electronics Store,Factory,Falafel Restaurant,Farm,Fast Food Restaurant,Fish Market,Fishing Spot,Fishing Store,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Forest,French Restaurant,Fruit & Vegetable Store,Furniture / Home Store,Garden Center,Gas Station,Gastropub,General Entertainment,General Travel,Gift Shop,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Harbor / Marina,Hardware Store,Health & Beauty Service,Herbs & Spices Store,Historic Site,History Museum,Home Service,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Kebab Restaurant,Lake,Liquor Store,Lounge,Market,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Mini Golf,Miscellaneous Shop,Motel,Motorsports Shop,Movie Theater,Moving Target,Multiplex,Museum,Music Venue,New American Restaurant,Nightclub,Opera House,Other Repair Shop,Outlet Store,Park,Pedestrian Plaza,Performing Arts Venue,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Pool,Pub,Racetrack,Ramen Restaurant,Real Estate Office,Record Shop,Recreation Center,Rental Car Location,Resort,Restaurant,River,Rock Club,Romanian Restaurant,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Ski Area,Soccer Field,Soccer Stadium,Soup Place,Souvenir Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Thai Restaurant,Theater,Theme Park,Tourist Information Center,Toy / Game Store,Track,Trail,Train Station,Travel & Transport,Tree,Tunnel,Vacation Rental,Vegetarian / Vegan Restaurant,Watch Shop,Water Park,Waterfall,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Women's Store,Zoo
0,Akkerhaugen,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0
1,Alstad,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.500000,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [37]:
norway_grouped.shape

(238, 198)

In [38]:
#Now, we are going to show what has more frequency in each of those neighborhoods.
num_top_venues = 5

for hood in norway_grouped['City']:
    print("----"+hood+"----")
    temp = norway_grouped[norway_grouped['City'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Akkerhaugen----
                   venue  freq
0             Waterfront   0.5
1                  Hotel   0.5
2    American Restaurant   0.0
3       Ramen Restaurant   0.0
4  Performing Arts Venue   0.0


----Alstad----
                 venue  freq
0        Grocery Store   0.5
1               Market   0.5
2  American Restaurant   0.0
3     Ramen Restaurant   0.0
4             Pharmacy   0.0


----Alta----
                 venue  freq
0                Hotel  0.50
1           Restaurant  0.25
2      Harbor / Marina  0.25
3  American Restaurant  0.00
4            Racetrack  0.00


----Andenes----
                 venue  freq
0        Boat or Ferry   1.0
1  American Restaurant   0.0
2     Ramen Restaurant   0.0
3             Pharmacy   0.0
4                 Pier   0.0


----Arendal----
              venue  freq
0              Café  0.18
1  Sushi Restaurant  0.09
2        Restaurant  0.09
3             Plaza  0.09
4             Hotel  0.09


----Asker----
              venue  freq
0     

In [39]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [40]:
#And add it into a pandas dataframe to be able to apply K-clustering.
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
cities_sorted = pd.DataFrame(columns=columns)
cities_sorted['City'] = norway_grouped['City']

for ind in np.arange(norway_grouped.shape[0]):
    cities_sorted.iloc[ind, 1:] = return_most_common_venues(norway_grouped.iloc[ind, :], num_top_venues)

cities_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Akkerhaugen,Waterfront,Hotel,Zoo,Flower Shop,Garden Center,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest,Food Truck
1,Alstad,Grocery Store,Market,Zoo,Food,Garden Center,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest,Food Truck
2,Alta,Hotel,Restaurant,Harbor / Marina,Zoo,Food,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest,Food Truck
3,Andenes,Boat or Ferry,Zoo,Food & Drink Shop,Gas Station,Garden Center,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest,Food Truck
4,Arendal,Café,Restaurant,Grocery Store,Hotel,Plaza,Sushi Restaurant,Chinese Restaurant,Tapas Restaurant,Train Station,Bar


In [41]:
# set number of clusters, in this case we choose 3 clusters because if its >3 we will have clusters with only one data in it.
kclusters = 4

norway_grouped_clustering = norway_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(norway_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 2, 0, 3, 3, 3, 3, 3, 0, 3], dtype=int32)

In [42]:
df.head()

,City,Latitude,Longitude
0,Oslo,59.916667,10.750000
1,Bergen,60.391111,5.324722
2,Stavanger,58.966667,5.750000
3,Trondheim,63.416667,10.416667
4,Drammen,59.742500,10.211667


In [43]:
# add clustering labels
cities_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

norway_merged = df

# merge netherlands_merged with cities_sorted to add latitude/longitude for each neighborhood
norway_merged = norway_merged.join(cities_sorted.set_index('City'), on='City')

norway_merged.head() # check the last columns!

,City,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Oslo,59.916667,10.750000,3.0,Bar,Beer Bar,Coffee Shop,Japanese Restaurant,Hotel,French Restaurant,Cocktail Bar,Mini Golf,Gym / Fitness Center,Indie Movie Theater
1,Bergen,60.391111,5.324722,3.0,Bar,Restaurant,Café,Hotel,Scandinavian Restaurant,Pub,Bakery,Sushi Restaurant,Coffee Shop,American Restaurant
2,Stavanger,58.966667,5.750000,2.0,Grocery Store,Diner,Watch Shop,Market,Pizza Place,Coffee Shop,Italian Restaurant,Auto Workshop,Food Service,Gastropub
3,Trondheim,63.416667,10.416667,2.0,Grocery Store,Pizza Place,Train Station,Zoo,Gas Station,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest,Food Truck
4,Drammen,59.742500,10.211667,3.0,American Restaurant,Plaza,Pizza Place,Movie Theater,Tapas Restaurant,Coffee Shop,Seafood Restaurant,Lounge,Bar,Chinese Restaurant


In [44]:
cities_sorted.head()

,Cluster Labels,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,Akkerhaugen,Waterfront,Hotel,Zoo,Flower Shop,Garden Center,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest,Food Truck
1,2,Alstad,Grocery Store,Market,Zoo,Food,Garden Center,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest,Food Truck
2,0,Alta,Hotel,Restaurant,Harbor / Marina,Zoo,Food,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest,Food Truck
3,3,Andenes,Boat or Ferry,Zoo,Food & Drink Shop,Gas Station,Garden Center,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest,Food Truck
4,3,Arendal,Café,Restaurant,Grocery Store,Hotel,Plaza,Sushi Restaurant,Chinese Restaurant,Tapas Restaurant,Train Station,Bar


#Deleting NaN Cluster Labels and making them int for ploting purposes in the future.
cities_sorted.shape
cities_sorted = cities_sorted.dropna(axis='rows')
cities_sorted['Cluster Labels'] = cities_sorted['Cluster Labels'].astype(int)
cities_sorted.head()

In [45]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=7)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(norway_merged['Latitude'], norway_merged['Longitude'], norway_merged['City'], cities_sorted['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [46]:
#Cluster 0 
norway_merged.loc[norway_merged['Cluster Labels'] == 0, norway_merged.columns[[0] + list(range(4, norway_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Narvik,Hotel,Bar,Café,Ski Area,Zoo,Food,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest
22,Alta,Hotel,Restaurant,Harbor / Marina,Zoo,Food,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest,Food Truck
40,Rosendal,Park,Hotel,Food,Gas Station,Garden Center,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest,Food Truck
111,Brattvåg,Hotel,Pizza Place,Zoo,Food,Garden Center,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest,Food Truck
125,Falkhytta,Hotel,Construction & Landscaping,Café,Zoo,Food,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest,Food Truck
129,Oteren,Hotel,Zoo,Flower Shop,Garden Center,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest,Food Truck,Food Service
142,Bogen,Hotel,Fast Food Restaurant,Zoo,Food,Garden Center,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest,Food Truck
208,Ulvik,Hotel,Campground,Zoo,Food,Garden Center,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest,Food Truck
243,Mo,Hotel,Zoo,Flower Shop,Garden Center,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest,Food Truck,Food Service
252,Stord,Hotel,Supermarket,Athletics & Sports,Food,Gas Station,Garden Center,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest


In [47]:
#Cluster 1 
norway_merged.loc[norway_merged['Cluster Labels'] == 1, norway_merged.columns[[0] + list(range(4, norway_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
201,Fauske,Bakery,Zoo,Food,Gas Station,Garden Center,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest,Food Truck
219,Rødberg,Bakery,Zoo,Food,Gas Station,Garden Center,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest,Food Truck
293,Tonstad,Bakery,Zoo,Food,Gas Station,Garden Center,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest,Food Truck
388,Vinje,Bakery,History Museum,Zoo,Food,Garden Center,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest,Food Truck


In [48]:
#Cluster 2 
norway_merged.loc[norway_merged['Cluster Labels'] == 2, norway_merged.columns[[0] + list(range(4, norway_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Stavanger,Grocery Store,Diner,Watch Shop,Market,Pizza Place,Coffee Shop,Italian Restaurant,Auto Workshop,Food Service,Gastropub
3,Trondheim,Grocery Store,Pizza Place,Train Station,Zoo,Gas Station,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest,Food Truck
7,Tomasjorda,Grocery Store,Boat or Ferry,Food Truck,Zoo,Food & Drink Shop,Garden Center,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest
21,Molde,Sporting Goods Shop,Grocery Store,Shopping Mall,Zoo,Flower Shop,Fruit & Vegetable Store,French Restaurant,Forest,Food Truck,Food Service
29,Finnsnes,Grocery Store,Thai Restaurant,Café,Gym,Shopping Mall,Zoo,Food,Fruit & Vegetable Store,French Restaurant,Forest
38,Tingvoll,Grocery Store,School,Hotel,Harbor / Marina,Zoo,Flower Shop,Fruit & Vegetable Store,French Restaurant,Forest,Food Truck
52,Hokksund,Construction & Landscaping,Grocery Store,Hardware Store,Zoo,Food,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest,Food Truck
53,Lurøy,Grocery Store,Hostel,Harbor / Marina,Zoo,Food,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest,Food Truck
84,Selbu,Grocery Store,Liquor Store,Zoo,Food,Garden Center,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest,Food Truck
99,Alstad,Grocery Store,Market,Zoo,Food,Garden Center,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest,Food Truck


In [49]:
#Cluster 3 
norway_merged.loc[norway_merged['Cluster Labels'] == 3, norway_merged.columns[[0] + list(range(4, norway_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Oslo,Bar,Beer Bar,Coffee Shop,Japanese Restaurant,Hotel,French Restaurant,Cocktail Bar,Mini Golf,Gym / Fitness Center,Indie Movie Theater
1,Bergen,Bar,Restaurant,Café,Hotel,Scandinavian Restaurant,Pub,Bakery,Sushi Restaurant,Coffee Shop,American Restaurant
4,Drammen,American Restaurant,Plaza,Pizza Place,Movie Theater,Tapas Restaurant,Coffee Shop,Seafood Restaurant,Lounge,Bar,Chinese Restaurant
6,Kristiansand,Brewery,Lake,Music Venue,Café,Pub,Farm,Food Service,Garden Center,Furniture / Home Store,Fruit & Vegetable Store
8,Ålesund,Hotel,Café,Coffee Shop,Restaurant,Fast Food Restaurant,Shopping Mall,Harbor / Marina,Gastropub,Seafood Restaurant,Clothing Store
9,Sandnes,Shopping Mall,Hotel,Sushi Restaurant,Bakery,Grocery Store,Coffee Shop,Train Station,Burger Joint,Chinese Restaurant,Steakhouse
10,Haugesund,Shopping Mall,Restaurant,Park,Hotel,Sushi Restaurant,Café,Fast Food Restaurant,Bakery,Soccer Stadium,Food
11,Tønsberg,Supermarket,Soccer Field,Pizza Place,Stadium,Electronics Store,Food & Drink Shop,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest
12,Moss,Bus Station,Restaurant,Stadium,Hotel,Sushi Restaurant,Café,Grocery Store,Shopping Mall,Beer Bar,Food & Drink Shop
13,Bodø,Hotel,Coffee Shop,Restaurant,Pizza Place,Brewery,Shopping Mall,Shoe Store,Seafood Restaurant,Café,Supermarket


In [50]:
#Cluster 3 
norway_merged.loc[norway_merged['Cluster Labels'] == 3, norway_merged.columns[[0] + list(range(4, norway_merged.shape[1]))]]

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Oslo,Bar,Beer Bar,Coffee Shop,Japanese Restaurant,Hotel,French Restaurant,Cocktail Bar,Mini Golf,Gym / Fitness Center,Indie Movie Theater
1,Bergen,Bar,Restaurant,Café,Hotel,Scandinavian Restaurant,Pub,Bakery,Sushi Restaurant,Coffee Shop,American Restaurant
4,Drammen,American Restaurant,Plaza,Pizza Place,Movie Theater,Tapas Restaurant,Coffee Shop,Seafood Restaurant,Lounge,Bar,Chinese Restaurant
6,Kristiansand,Brewery,Lake,Music Venue,Café,Pub,Farm,Food Service,Garden Center,Furniture / Home Store,Fruit & Vegetable Store
8,Ålesund,Hotel,Café,Coffee Shop,Restaurant,Fast Food Restaurant,Shopping Mall,Harbor / Marina,Gastropub,Seafood Restaurant,Clothing Store
9,Sandnes,Shopping Mall,Hotel,Sushi Restaurant,Bakery,Grocery Store,Coffee Shop,Train Station,Burger Joint,Chinese Restaurant,Steakhouse
10,Haugesund,Shopping Mall,Restaurant,Park,Hotel,Sushi Restaurant,Café,Fast Food Restaurant,Bakery,Soccer Stadium,Food
11,Tønsberg,Supermarket,Soccer Field,Pizza Place,Stadium,Electronics Store,Food & Drink Shop,Furniture / Home Store,Fruit & Vegetable Store,French Restaurant,Forest
12,Moss,Bus Station,Restaurant,Stadium,Hotel,Sushi Restaurant,Café,Grocery Store,Shopping Mall,Beer Bar,Food & Drink Shop
13,Bodø,Hotel,Coffee Shop,Restaurant,Pizza Place,Brewery,Shopping Mall,Shoe Store,Seafood Restaurant,Café,Supermarket
